In [2]:
import numpy as np 
import pandas as pd 
import os
import glob
import json
from matplotlib import pyplot as plt
import seaborn as sns
from collections import Counter
import gc
from PIL import Image

%matplotlib inline

In [3]:
os.getcwd()

'C:\\Users\\Meehir\\Documents\\GitHub\\project-5'

In [4]:
kaggle_20_path = "D:\\METIS\\kaggle\\imet-2020-fgvc7\\"
kaggle_19_path = "D:\\METIS\\kaggle\\imet-2019-fgvc6\\"

In [5]:
train_20_df = pd.read_csv(kaggle_20_path + 'train.csv')
test_20_df = pd.read_csv(kaggle_20_path + 'sample_submission.csv')
labels_20_df = pd.read_csv(kaggle_20_path + 'labels.csv')

train_19_df = pd.read_csv(kaggle_19_path + 'train.csv')
test_19_df = pd.read_csv(kaggle_19_path + 'sample_submission.csv')
labels_19_df = pd.read_csv(kaggle_19_path + 'labels.csv')

In [33]:
labels_20_df

,attribute_name,labels,type,label
attribute_id,,,,
0,country::afghanistan,country::afghanistan,country,afghanistan
1,country::alamania,country::alamania,country,alamania
2,country::algeria,country::algeria,country,algeria
3,country::arabia,country::arabia,country,arabia
4,country::armenia,country::armenia,country,armenia
...,...,...,...,...
3469,tags::writing,tags::writing,tags,writing
3470,tags::writing implements,tags::writing implements,tags,writing implements
3471,tags::zebras,tags::zebras,tags,zebras


In [6]:
train_20_df

,id,attribute_ids
0,000040d66f14ced4cdd18cd95d91800f,448 2429 782
1,0000ef13e37ef70412166725ec034a8a,2997 3231 2730 3294 3099 2017 784
2,0001eeb4a06e8daa7c6951bcd124c3c7,2436 1715 23
3,000226398d224de78b191e6db45fd94e,2997 3433 448 782
4,00029c3b0171158d63b1bbf803a7d750,3465 3322 3170 1553 781
...,...,...
142114,fffd00c472c1856f5c8b7e41311e2344,125 783 96 27
142115,fffdb3cde6cfd3b335fc56ecf2f1eae5,2732 3170 448 1559 785
142116,fffdde4325edb076dccea37978c1378a,2813 233 1561 785
142117,fffe25dccc93a5e9e12f26d494dae26f,3095 408 1610 1470 1899 1449


In [31]:
df_out = train_20_df.assign(**pd.get_dummies(train_20_df['attribute_ids'].apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True)).sum(level=0))
df_out

AttributeError: 'bool' object has no attribute 'any'

In [24]:
train_20_df['attribute_ids'][0]

'448 2429 782'

In [34]:
train_20_df.head()

,id,attribute_ids
0,000040d66f14ced4cdd18cd95d91800f,448 2429 782
1,0000ef13e37ef70412166725ec034a8a,2997 3231 2730 3294 3099 2017 784
2,0001eeb4a06e8daa7c6951bcd124c3c7,2436 1715 23
3,000226398d224de78b191e6db45fd94e,2997 3433 448 782
4,00029c3b0171158d63b1bbf803a7d750,3465 3322 3170 1553 781


In [8]:
labels_20_df

,attribute_id,attribute_name
0,0,country::afghanistan
1,1,country::alamania
2,2,country::algeria
3,3,country::arabia
4,4,country::armenia
...,...,...
3469,3469,tags::writing
3470,3470,tags::writing implements
3471,3471,tags::zebras
3472,3472,tags::zeus


In [9]:
labels_20_df['labels'] = labels_20_df['attribute_name'].copy()
labels_19_df['labels'] = labels_19_df['attribute_name'].copy()

In [10]:
labels_20_df['type'] = labels_20_df['labels'].apply(lambda x: x.split("::")[0])
labels_20_df['label'] = labels_20_df['labels'].apply(lambda x: x.split("::")[1])

labels_19_df['type'] = labels_19_df['labels'].apply(lambda x: x.split("::")[0])
labels_19_df['label'] = labels_19_df['labels'].apply(lambda x: x.split("::")[1])

In [11]:
labels_20_df.set_index('attribute_id', inplace=True)
labels_19_df.set_index('attribute_id', inplace=True)

In [12]:
dummies_20 = pd.get_dummies(labels_20_df['type'])
dummies_20

,country,culture,dimension,medium,tags
attribute_id,,,,,
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
3469,0,0,0,0,1
3470,0,0,0,0,1
3471,0,0,0,0,1


In [13]:
label_types = list(set(labels_20_df['type'].to_list()))
d = {}
for label in label_types:
    d[label] = labels_20_df[labels_20_df['type'] == label]['label'].to_list()
d.keys()

dict_keys(['country', 'medium', 'dimension', 'culture', 'tags'])

In [15]:
countries = d['country']
print(countries)

['afghanistan', 'alamania', 'algeria', 'arabia', 'armenia', 'austria', 'azerbaijan', 'belgium', 'borneo', 'bulgaria', 'byzantine egypt', 'canada', 'central italy', 'chile', 'china', 'colombia', 'colombia or ecuador', 'costa rica', 'costa rica or panama', 'czech republic', 'democratic republic of the congo', 'dominican republic', 'ecuador', 'egypt', 'egypt and sudan', 'egypt or iraq', 'egypt or syria', 'england', 'ethiopia', 'france', 'france or germany', 'french', 'gabon', 'gaul', 'germany', 'ghana', 'greece', 'guatemala', 'guatemala or mexico', 'guyana', 'honduras', 'hungary', 'india', 'india or iran', 'indonesia', 'iran', 'iran (persia)', 'iran or central asia', 'iran or iraq', 'iran or present-day uzbekistan', 'iran or turkey', 'iraq', 'iraq or syria', 'ireland', 'italy', 'japan', 'kenya', 'korea', 'madagascar', 'mali', 'mexico', 'morocco', 'mozambique', 'netherlands', 'new zealand', 'nigeria', 'north netherlands', 'northern france', 'northwestern iran or baghdad', 'nubia (sudan)', 

In [14]:
fix_countries = ['byzantine egypt', 'central italy', 'colombia or ecuador', 'costa rica or panama', 'egypt and sudan', 'egypt or iraq', 
                 'egypt  or syria', 'france or germany', 'french', 'gaul', 'guatamala or mexico','india or iran', 'iran (persia)', 
                 'iran or central asia', 'iran or iraq', 'iran or present-day uzbekistan', 'iran or turkey', 'iraq or syria', 
                 'north netherlands', 'northern france','northwest iran or baghdad', 'nubia (sudan)', 'present-day afghanistan', 'present-day azerbaijan', 
                 'present-day france', 'present-day greece', 'present-day pakistan', 'present-day uzbekistan',
                'south netherlands', 'western turkenistan']

In [19]:
egypt_list = []
for country in countries:
    if 'egypt' in country:
        egypt_list.append(country)
egypt_list

['byzantine egypt',
 'egypt',
 'egypt and sudan',
 'egypt or iraq',
 'egypt or syria']

if true, add into column
if false, new column